<a href="https://colab.research.google.com/github/renatavel/goit_ds_homeworks/blob/main/hw_13_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras
import matplotlib.pyplot as plt

In [18]:
from keras import datasets
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [16]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [19]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [20]:
x_train = tf.repeat(x_train, 3, axis=-1)
x_test = tf.repeat(x_test, 3, axis=-1)

In [21]:
x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))
x_train.shape

TensorShape([60000, 32, 32, 3])

In [22]:
y_train_hot = keras.utils.to_categorical(y_train, 10)
y_test_hot = keras.utils.to_categorical(y_test, 10)

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [24]:
from keras import applications
model_base = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

In [25]:
for layer in model_base.layers:
    layer.trainable = False

In [26]:
for layer in model_base.layers[-4:]:
    layer.trainable = True

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D

model = Sequential([
    model_base,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

In [28]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint_fine_tuning = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [30]:
history = model.fit(
    datagen.flow(x_train, y_train_hot, batch_size=64),
    epochs=20,
    validation_data=(x_test, y_test_hot),
    callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=5),
        checkpoint_fine_tuning
    ]
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.2364 - loss: 2.3656
Epoch 1: val_accuracy improved from -inf to 0.38390, saving model to best_model.keras
938/938 ━━━━━━━━━━━━━━━━━━━━ 58s 52ms/step - accuracy: 0.2364 - loss: 2.3652 - val_accuracy: 0.3839 - val_loss: 1.6144
Epoch 2/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4130 - loss: 1.5215
Epoch 2: val_accuracy improved from 0.38390 to 0.48300, saving model to best_model.keras
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.4130 - loss: 1.5214 - val_accuracy: 0.4830 - val_loss: 1.2049
Epoch 3/20
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4827 - loss: 1.2143
Epoch 3: val_accuracy improved from 0.48300 to 0.55140, saving model to best_model.keras
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.4828 - loss: 1.2142 - val_accuracy: 0.5514 - val_loss: 1.0375
Epoch 4/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6001 - loss: 0.9500
Epoch 4: val_accuracy improved from 0.5514

In [31]:
best_model = keras.models.load_model('best_model.keras')
test_loss, test_accuracy = best_model.evaluate(x_test, y_test_hot)
print(f"Accuracy on the testing data: {test_accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8993 - loss: 0.2915
Accuracy on the testing data: 89.87%
